# 🏭 RetailNova - Enterprise Data Platform
## Notebook 01: Bronze Layer Ingestion Demo

This notebook demonstrates the full Bronze ingestion pipeline:
1. Connect to SQL Server (on-prem simulation)
2. Read watermark for incremental extraction
3. Extract changed records
4. Add audit columns
5. Write to MinIO (Delta format)
6. Update watermark


In [ ]:
import sys
sys.path.insert(0, '/home/jovyan')

from pipelines.spark_session import build_spark_session
from pipelines.config import source_config, storage_config, metadata_config

# Build SparkSession (connects to Docker Spark cluster)
spark = build_spark_session('RetailNova-Notebook-Bronze')
print('✓ Spark session created')
print(f'  Spark version: {spark.version}')
print(f'  Master: {spark.sparkContext.master}')

In [ ]:
# ── Step 1: Check what's in the source SQL Server ─────────────────────────
from pipelines.config import source_config

df_customers = (
    spark.read.format('jdbc')
    .option('url',      source_config.jdbc_url)
    .option('dbtable',  'dbo.customers')
    .option('user',     source_config.username)
    .option('password', source_config.password)
    .option('driver',   source_config.jdbc_properties['driver'])
    .load()
)

print(f'Customers in SQL Server: {df_customers.count()}')
df_customers.show(5)

In [ ]:
# ── Step 2: Check watermark state ─────────────────────────────────────────
import psycopg2
from pipelines.config import metadata_config

conn = psycopg2.connect(
    host=metadata_config.host,
    port=metadata_config.port,
    dbname=metadata_config.database,
    user=metadata_config.username,
    password=metadata_config.password,
)
with conn.cursor() as cur:
    cur.execute('SELECT * FROM pipeline_watermarks')
    print('Current watermarks:')
    for row in cur.fetchall():
        print(f'  {row[0]:30} -> last_watermark={row[1]}')
conn.close()

In [ ]:
# ── Step 3: Run Bronze Ingestion for a single table ───────────────────────
from pipelines.bronze_ingestion import run_bronze_ingestion

# Run only for 'customers' table first
run_bronze_ingestion(spark, tables=['customers'])

In [ ]:
# ── Step 4: Read back from Bronze Delta ───────────────────────────────────
from pipelines.config import storage_config

bronze_path = storage_config.layer_path('bronze', 'customers')
print(f'Reading from: {bronze_path}')

df_bronze = spark.read.format('delta').load(bronze_path)
print(f'Bronze customers count: {df_bronze.count()}')
df_bronze.printSchema()
df_bronze.show(5, truncate=False)

In [ ]:
# ── Step 5: Show Delta table history (time travel) ────────────────────────
from delta.tables import DeltaTable

dt = DeltaTable.forPath(spark, bronze_path)
dt.history().show(10, truncate=False)

In [ ]:
# ── Step 6: Run ALL tables ────────────────────────────────────────────────
run_bronze_ingestion(spark)  # No tables param = runs all
print('\n✓ All Bronze tables loaded')